## 📑Оглавление
- [Библиотеки](#Библиотеки)
- [Предобработка](#Предобработка)

## Библиотеки

In [2]:
import os
from zipfile import ZipFile
from datetime import date

import pandas as pd
import numpy as np

## Предобработка

In [3]:
general_path = 'archive.zip' # путь к архиву

In [4]:
with ZipFile(general_path, 'r') as archive: # открываем архив
    df_coins = None # Общий датафрейм
    for name_file in archive.namelist(): # бегаем по именам файлов, содержащихся в архиве
        if name_file.endswith('.csv'): # имеет расширение csv
            with archive.open(name_file, 'r') as file_coin: # открываем файл внутри архива
                df_coin = pd.read_csv(file_coin) # считываем файл по монете в датафрейм
                if df_coins is None: # если общий датафрейм еще не создан, то он будет ссылаться на первую монету
                    df_coins = df_coin
                else: # затем будет прибавлять следующую монету
                    df_coins = pd.concat([df_coins, df_coin], ignore_index=True)
display(df_coins)

,SNo,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap
0,1,Aave,AAVE,2020-10-05 23:59:59,55.112358,49.787900,52.675035,53.219243,0.000000e+00,8.912813e+07
1,2,Aave,AAVE,2020-10-06 23:59:59,53.402270,40.734578,53.291969,42.401599,5.830915e+05,7.101144e+07
2,3,Aave,AAVE,2020-10-07 23:59:59,42.408314,35.970690,42.399947,40.083976,6.828342e+05,6.713004e+07
3,4,Aave,AAVE,2020-10-08 23:59:59,44.902511,36.696057,39.885262,43.764463,1.658817e+06,2.202651e+08
4,5,Aave,AAVE,2020-10-09 23:59:59,47.569533,43.291776,43.764463,46.817744,8.155377e+05,2.356322e+08
...,...,...,...,...,...,...,...,...,...,...
37077,2889,XRP,XRP,2021-07-02 23:59:59,0.667287,0.634726,0.659890,0.656763,2.061607e+09,3.030759e+10
37078,2890,XRP,XRP,2021-07-03 23:59:59,0.683677,0.644653,0.655639,0.672888,1.872820e+09,3.105172e+10
37079,2891,XRP,XRP,2021-07-04 23:59:59,0.707783,0.665802,0.673218,0.694945,1.885242e+09,3.206960e+10
37080,2892,XRP,XRP,2021-07-05 23:59:59,0.695653,0.648492,0.695653,0.654300,2.076373e+09,3.019395e+10


In [5]:
df_coins.drop(columns=['SNo'], inplace=True) # Убираем колонку с номером

In [6]:
coins = list(df_coins['Name'].unique()) # список представленных монет
# разобьем эти монеты по сегментам
coin_segments = [
    ['Aave', 'DeFi'],
    ['Binance Coin', 'Exchange'],
    ['Bitcoin', 'Layer1'],
    ['Cardano', 'Layer1'],
    ['Chainlink', 'DeFi'],
    ['Cosmos', 'Layer0'],
    ['Crypto.com Coin', 'Exchange'],
    ['Dogecoin', 'Payments'],
    ['EOS', 'Layer1'],
    ['Ethereum', 'Layer1'],
    ['IOTA', 'Infrastructure'],
    ['Litecoin', 'Payments'],
    ['Monero', 'Privacy'],
    ['NEM', 'Layer1'],
    ['Polkadot', 'Layer0'],
    ['Solana', 'Layer1'],
    ['Stellar', 'Payments'],
    ['Tether', 'Stablecoin'],
    ['TRON', 'Layer1'],
    ['USD Coin', 'Stablecoin'],
    ['Uniswap', 'DeFi'],
    ['Wrapped Bitcoin', 'Bridge'],
    ['XRP', 'Payments']
]
# создадим датафрейм на основе списка списков
df_segments = pd.DataFrame(coin_segments, columns=['Name', 'Segment'])
display(df_segments.head())

,Name,Segment
0,Aave,DeFi
1,Binance Coin,Exchange
2,Bitcoin,Layer1
3,Cardano,Layer1
4,Chainlink,DeFi


In [7]:
# присоединяем колонку с сегментами к нашему датафрейму
df_origin = df_coins.merge(df_segments, how='left', on='Name')

# наш датафрейм готов к работе
display(df_origin.head())

,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap,Segment
0,Aave,AAVE,2020-10-05 23:59:59,55.112358,49.787900,52.675035,53.219243,0.000000e+00,8.912813e+07,DeFi
1,Aave,AAVE,2020-10-06 23:59:59,53.402270,40.734578,53.291969,42.401599,5.830915e+05,7.101144e+07,DeFi
2,Aave,AAVE,2020-10-07 23:59:59,42.408314,35.970690,42.399947,40.083976,6.828342e+05,6.713004e+07,DeFi
3,Aave,AAVE,2020-10-08 23:59:59,44.902511,36.696057,39.885262,43.764463,1.658817e+06,2.202651e+08,DeFi
4,Aave,AAVE,2020-10-09 23:59:59,47.569533,43.291776,43.764463,46.817744,8.155377e+05,2.356322e+08,DeFi


In [8]:
df_origin['Date'] = pd.to_datetime(df_origin['Date']) # преобразуем тип данных колонки Date в тип datetime64[ns]

In [9]:
df_origin['Date'] = df_origin['Date'].dt.floor('D') # убираем время 23:59:59 из столбца, сохраняя тип datetime64[ns]

In [10]:
# преобразуем наименования колонок в нижний регистр
df_origin.columns = [name_col.lower() for name_col in list(df_origin.columns)] 

In [13]:
df_origin.to_csv('clean_data.csv', index=False, encoding='utf-8-sig')